# PPO for Portfolio Management
This tutorial is to demonstrate an example of using PPO to do portfolio management

## Step1: Import Packages

In [2]:

import warnings
warnings.filterwarnings("ignore")
import sys
from pathlib import Path
import os
import torch

ROOT = os.path.dirname(os.path.abspath("."))
sys.path.append(ROOT)

import argparse
import os.path as osp
from mmcv import Config
from trademaster.utils import replace_cfg_vals
from trademaster.nets.builder import build_net
from trademaster.environments.builder import build_environment
from trademaster.datasets.builder import build_dataset
from trademaster.agents.builder import build_agent
from trademaster.optimizers.builder import build_optimizer
from trademaster.losses.builder import build_loss
from trademaster.trainers.builder import build_trainer

## Step2: Import Configs

In [3]:
parser = argparse.ArgumentParser(description='Download Alpaca Datasets')
parser.add_argument("--config", default=osp.join(ROOT, "configs", "portfolio_management", "portfolio_management_exchange_ppo_ppo_adam_mse.py"),
                    help="download datasets config file path")
parser.add_argument("--task_name", type=str, default="train")
args, _ = parser.parse_known_args()

cfg = Config.fromfile(args.config)
task_name = args.task_name
cfg = replace_cfg_vals(cfg)


## Step3: Build Dataset

In [4]:
dataset = build_dataset(cfg)


## Step4: Build Trainer

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
work_dir = os.path.join(ROOT, cfg.trainer.work_dir)

if not os.path.exists(work_dir):
    os.makedirs(work_dir)
cfg.dump(osp.join(work_dir, osp.basename(args.config)))

trainer = build_trainer(cfg, default_args=dict(dataset=dataset, device = device))

| Arguments Keep work_dir: E:\workspace\RA\TradeMaster\work_dir/portfolio_management_exchange_ppo_ppo_adam_mse


## Step5: Train, Valid and Test

In [6]:
trainer.train_and_valid()


2023-02-28 01:08:34,116	INFO trainer.py:2321 -- Executing eagerly (framework='tf2'), with eager_tracing=False. For production workloads, make sure to set eager_tracing=True  in order to match the speed of tf-static-graph (framework='tf'). For debugging purposes, `eager_tracing=False` is the best choice.
2023-02-28 01:08:34,117	INFO trainer.py:903 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=20440) D:\ProgramData\Anaconda3\envs\python3.9\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
(RolloutWorker pid=20440)   warnings.warn(
(RolloutWorker pid=19936) D:\ProgramData\An

Train Episode: [1/20]


2023-02-28 01:09:09,690	WARNING deprecation.py:46 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!
2023-02-28 01:09:09,691	WARNING deprecation.py:46 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!


Valid Episode: [1/20]
+--------------+-------------+------------+--------------+
| Total Return | Sharp Ratio | Volatility | Max Drawdown |
+--------------+-------------+------------+--------------+
| -49.961655%  |  -90.971991 | 0.389892%  |  50.212797%  |
+--------------+-------------+------------+--------------+
Train Episode: [2/20]
Valid Episode: [2/20]
+--------------+-------------+------------+--------------+
| Total Return | Sharp Ratio | Volatility | Max Drawdown |
+--------------+-------------+------------+--------------+
| -50.680183%  |  -94.262338 | 0.381694%  |  50.931566%  |
+--------------+-------------+------------+--------------+
Train Episode: [3/20]
(RolloutWorker pid=20440) +--------------+-------------+------------+--------------+
(RolloutWorker pid=20440) | Total Return | Sharp Ratio | Volatility | Max Drawdown |
(RolloutWorker pid=20440) +--------------+-------------+------------+--------------+
(RolloutWorker pid=20440) | -99.838413%  |  -62.103237 | 0.402705% 

In [10]:
trainer.test()

(RolloutWorker pid=23324) D:\ProgramData\Anaconda3\envs\python3.9\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
(RolloutWorker pid=23324)   warnings.warn(
(RolloutWorker pid=24012) D:\ProgramData\Anaconda3\envs\python3.9\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
(RolloutWorker pid=24012)   warnings.warn(
(RolloutWorker

RayActorError: The actor died because of an error raised in its creation task, [36mray::RolloutWorker.__init__()[39m (pid=24012, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x00000237D1B0BBB0>)
  File "D:\ProgramData\Anaconda3\envs\python3.9\lib\site-packages\gym\envs\registration.py", line 156, in make
    return registry.make(id, **kwargs)
  File "D:\ProgramData\Anaconda3\envs\python3.9\lib\site-packages\gym\envs\registration.py", line 100, in make
    spec = self.spec(path)
  File "D:\ProgramData\Anaconda3\envs\python3.9\lib\site-packages\gym\envs\registration.py", line 129, in spec
    raise error.Error('Attempted to look up malformed environment ID: {}. (Currently all IDs must be of the form {}.)'.format(id.encode('utf-8'), env_id_re.pattern))
gym.error.Error: Attempted to look up malformed environment ID: b'portfolio_management'. (Currently all IDs must be of the form ^(?:[\w:-]+\/)?([\w:.-]+)-v(\d+)$.)

During handling of the above exception

(RolloutWorker pid=24012) 2023-02-28 01:40:53,855	ERROR worker.py:451 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=24012, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x00000237D1B0BBB0>)
(RolloutWorker pid=24012)   File "D:\ProgramData\Anaconda3\envs\python3.9\lib\site-packages\gym\envs\registration.py", line 156, in make
(RolloutWorker pid=24012)     return registry.make(id, **kwargs)
(RolloutWorker pid=24012)   File "D:\ProgramData\Anaconda3\envs\python3.9\lib\site-packages\gym\envs\registration.py", line 100, in make
(RolloutWorker pid=24012)     spec = self.spec(path)
(RolloutWorker pid=24012)   File "D:\ProgramData\Anaconda3\envs\python3.9\lib\site-packages\gym\envs\registration.py", line 129, in spec
(RolloutWorker pid=24012)     raise error.Error('Attempted to look up malformed environment ID: {}. (Currently all IDs must be of the form {}.)'.format(id.e